---
title: "A Comparative Benchmarking Study of Monte-Carlo, Black-Scholes-Merton and Heston Models for Equity Option Pricing"
format:
  pdf:
    toc: true
    toc-depth: 3
    geometry:
      - margin=15mm
---

## Section 1: Introduction and Summary

**1.1. Project Overview and Objective**

Equity option pricing remains a foundational problem in quantitative finance, with widespread practical applications spanning trading, risk management, and regulatory capital assessment. Despite decades of academic development, practitioners continue to rely on a small set of canonical models, each embodying distinct assumptions regarding asset dynamics, volatility behavior, and distributional properties of returns.

The objective of this study is to systematically benchmark three widely used equity option pricing frameworks — Monte-Carlo simulation, Black-Scholes-Merton (BSM), and the Heston stochastic volatility model — with the explicit aim of documenting their relative strengths, weaknesses, and practical limitations under realistic data and governance constraints. The benchmarking exercise is conducted using two parallel datasets: a proxy-based dataset designed for model development and regulatory defensibility, and an ETF-based dataset used for external price comparison and realism checks.

Rather than proposing a novel pricing model, this research focuses on comparative robustness, distributional realism, and out-of-sample behavior, reflecting the considerations faced by risk and model validation teams in institutional settings.


**1.2. Overview of the Models Considered**

The three models examined in this study span a spectrum of theoretical assumptions and practical complexity:

- Black-Scholes-Merton (BSM) represents the classical closed-form solution for European option pricing, assuming log-normally distributed asset prices, constant volatility, frictionless markets, and continuous hedging. Despite well-documented empirical violations of these assumptions, BSM remains an industry benchmark due to its interpretability, analytical tractability, and historical adoption.

- The Heston stochastic volatility model extends the BSM framework by introducing a mean-reverting stochastic process for volatility, thereby capturing volatility clustering and leverage effects observed in real markets. While more flexible than BSM, the Heston model introduces calibration complexity, numerical stability considerations, and additional parameter uncertainty.

- Monte-Carlo simulation, implemented in this study using a bootstrap-based, non-parametric approach, avoids strong parametric assumptions about return distributions. Instead, it relies on empirically observed return dynamics to generate future price paths, allowing for skewness, excess kurtosis, and regime-specific behavior to be preserved in simulated outcomes.

Together, these models enable a structured comparison between analytical tractability, stochastic realism, and distribution-free empirical modeling.


**1.3. Data Architecture and Dual-ETL Design**

A key design feature of this project is the construction of two distinct ETL pipelines, motivated by data availability constraints and regulatory considerations.

- The proxy-based dataset is constructed using liquid, publicly available equity and index data to create synthetic underlying assets suitable for model development, stress testing, and out-of-sample evaluation. This pipeline emphasizes transparency, reproducibility, long historical coverage, and suitability for statistical testing and regulatory defense. The proxy dataset serves as the primary foundation for model calibration, simulation, and risk evaluation.

- The ETF-based dataset sources option-relevant price information derived from exchange-traded funds tracking the same or closely related underlying assets. This pipeline is used exclusively for external price comparison, visual benchmarking, and plausibility checks of model-implied option prices. Due to limitations associated with free and publicly accessible option datasets, the ETF-based pipeline is not used for model calibration or statistical inference.


**1.4. Statistical Diagnostics and Distributional Testing**

Prior to model implementation, extensive statistical diagnostics are applied to the proxy-based dataset to assess the validity of common modeling assumptions. These diagnostics include:

- Tests for normality and log-normality of returns
- Evaluation of skewness and excess kurtosis
- Analysis of volatility clustering and regime dependence

Empirical evidence of non-Gaussian return behavior, fat tails, and asymmetry motivates the inclusion of a bootstrap-based Monte-Carlo framework and informs the interpretation of results obtained from parametric models such as BSM and Heston.


**1.5. Out-of-Sample Evaluation Framework**

Model evaluation is conducted using a clearly defined out-of-sample (OOS) framework, designed to assess robustness rather than in-sample fit. OOS parameters are selected to capture multiple dimensions of model performance, including:

- Pricing stability across market regimes
- Tail risk behavior and downside sensitivity
- Risk decomposition and factor exposure consistency

The evaluation framework emphasizes comparative behavior under identical inputs, ensuring that differences in outcomes can be attributed to model structure rather than data artifacts.


**1.6. Implementation Methodology (Summary)**

The overall implementation methodology follows a structured and reproducible sequence:

- Construction of proxy-based and ETF-based ETL pipelines
- Statistical diagnostics and assumption testing on proxy data
- Independent implementation of Monte-Carlo, BSM, and Heston models
- Model calibration and numerical validation, where applicable
- ETF-based benchmarking for external pricing realism
- Proxy-based out-of-sample risk and stability evaluation
- Comparative analysis across all models and evaluation metrics


**1.7. Summary of Findings**

**TO BE FILLED IN THE END**


**1.8. Compliance with SR 11-7 Model Risk and Governance Framework**

**TO BE FILLED IN THE END**


**Disclaimer**

This document is intended solely for academic, educational, and research purposes. The models, methodologies, data sources, and results presented herein are illustrative in nature and are not intended to constitute investment advice, trading recommendations, or financial forecasts. Any references to market instruments, prices, or returns are for analytical demonstration only. The analyses rely on publicly available data and simplifying assumptions, and may not fully capture real-world market frictions, liquidity constraints, transaction costs, or institutional trading considerations. This document does not intend to provide representations regarding the suitability of any model or result for live trading, risk management, or regulatory capital determination.

In [24]:
# Import necessary packages
import os, time, requests
from dotenv import load_dotenv
from contextlib import contextmanager
from IPython.display import display
import pandas as pd, numpy as np, matplotlib.pyplot as plt, yfinance as yf

# Initializing environment
load_dotenv()
zero_coupon = os.getenv('zero_coupon')
proxy_master = os.getenv('proxy_master')
etf_master = os.getenv('etf_master')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)
pd.set_option('display.float_format', '{:.4f}'.format)

# Defining time context block
@contextmanager
def time_block(label: str='Block'):
    start_time = time.perf_counter()
    try:
        yield
    finally:
        end_time = time.perf_counter()
        runtime = end_time - start_time
        print(f'{label} execution time: {runtime/60:.2f} minutes.')


## Section 2: Implementation of ETL Pipelines

**1.1. Overview**

This project adopts a dual data-architecture strategy to balance **economic correctness**, **empirical validation**, and **data accessibility constraints**:

- **Proxy-based architecture**:
  This architecture constructs a synthetic yet economically grounded state-space of contingent claims, designed to support model development, assumption testing, and controlled out-of-sample (OOS) evaluation. The approach deliberately decouples contract specification from market-quoted option chains by defining options across fixed grids of moneyness, maturity, and payoff type, while anchoring all valuations to observed underlying prices, realized volatility proxies, risk-free term structures, and dividend yield assumptions. By holding the contract space and input state variables constant, the framework enables clean comparative benchmarking of pricing models, ensuring that observed differences in valuation and risk metrics are attributable to model dynamics rather than data availability, liquidity effects, or market microstructure noise. This proxy-based design emphasizes transparency, economic interpretability, numerical stability, and robustness under model governance and regulatory scrutiny, consistent with principles outlined in formal model-risk management frameworks.

- **ETF-based architecture**: 
  This architecture leverages exchange-traded fund (ETF) option chains as market-anchored representations of contingent claims, designed to support empirical validation, calibration realism, and external consistency checks against observed option prices. The approach preserves the contract specifications embedded in traded ETF options, including listed strikes, standardized maturities, and observed call/put structures, while anchoring valuations to market-implied prices, bid–ask dynamics, and prevailing liquidity conditions. As a result, the ETF-based framework reflects the joint influence of investor demand, hedging activity, volatility risk premia, and market microstructure effects inherent in real-world options markets. By operating directly on quoted option chains, the architecture enables assessment of absolute pricing accuracy, calibration stability, and model fit to observed market prices, complementing the controlled benchmarking enabled by the proxy-based framework. Differences between model-implied and market-observed prices can therefore be interpreted in the context of model assumptions, volatility dynamics, and unmodeled risk premia, rather than purely synthetic contract design. This ETF-based design emphasizes market realism, empirical validity, and external benchmark alignment, providing a necessary counterbalance to synthetic architectures and supporting robust model validation, sensitivity analysis, and governance-oriented performance assessment when used in conjunction with controlled proxy-based evaluations.

- **Why a dual-architecture setup**:
  The dual-architecture design addresses the absence of licensed market data feeds (e.g., Bloomberg, Refinitiv, Eurex) while maintaining methodological rigor consistent with institutional model development practices. Early stages of model development—including assumption testing, sensitivity analysis, and numerical stability checks—require controlled experimentation, isolation of effects, and a repeatable state-space. These requirements are satisfied by the proxy-based architecture, but cannot be reliably met using a purely ETF-based framework due to changing contract availability, liquidity effects, and embedded market risk premia. In contrast, later stages of model calibration and out-of-sample (OOS) validation benefit from direct exposure to market-quoted option prices, where an ETF-based architecture provides a realistic representation of observed pricing dynamics that a proxy-based framework cannot replicate. This separation is consistent with formal model-risk management guidance. In particular, SR 11-7 states that *“model validation should employ a combination of theoretical evaluation, controlled testing, and benchmarking against observed outcomes.”* The dual-architecture setup operationalizes this principle by explicitly separating controlled model diagnostics from market-anchored validation, ensuring both interpretability and empirical relevance.

>**Note**: To deliberately eliminate confounding effects - attributable to cross-asset differences arising from leveraging different indices under proxy and ETF architecture - both architectures use the same EUR-denominated underlying (EXSA.DE - iShares Core EURO STOXX 50 UCITS, ETF, EUR, Xetra). The proxy-based architecture generates synthetic option contracts and model-implied prices to enable controlled comparison of model dynamics under consistent inputs. The ETF-based architecture uses observed option quotes on the same underlying to provide empirical outcomes analysis, acknowledging market microstructure noise, discrete dividend effects, and liquidity filters.


**1.2. ETL Pipeline**

In alignment with the SR 11-7 Model Risk Management framework, the project implements the following standardized ETL pipeline for both data architectures:

- **Step 1: Data Acquisition:** Sourcing raw market observables from documented, reproducible open-source channels.

- **Step 2: Data Ingestion:** Structured loading into analysis-ready data containers with version control and timestamping.

- **Step 3: Data Cleansing:** Handling missing values, calendar alignment, corporate action adjustments, and data sanity checks.

- **Step 4: Feature Engineering (as required):** Construction of derived quantities such as log returns, realized volatility measures, and term-structure interpolations.

- **Step 5: Rendering Model-Ready Datasets:** Final transformation into inputs compatible with Black–Scholes–Merton, Heston, and Monte Carlo pricing frameworks.

This pipeline separation ensures traceability, auditability, and reproducibility—key SR 11-7 requirements.


**1.3. Data Acquisition Sources and Rationale — Proxy-Based Architecture**

Under the proxy-based architecture, the following data components are sourced to construct a **theoretically consistent index-level pricing environment**. These inputs are used for model calibration, assumption testing, and controlled comparative analysis rather than direct replication of observed option prices.

Main Area 1: Underlying Index Prices - *Used to compute log returns, realized volatility, and to define the underlying state variable.*

- Daily closing prices - *Source: Yahoo Finance – EXSA.DE (iShares Core EURO STOXX 50 UCITS ETF, EUR, Xetra)*  
- Trading calendar and date alignment - *Derived from ETF price time series*

Main Area 2: Realized Volatility Proxy - *Used to characterize historical volatility dynamics and provide data-driven inputs and initial conditions for stochastic volatility models.*

- Rolling realized volatility (21 days) - *Engineered from log returns derived in Main Area 1*  
- Rolling realized volatility (63 days) - *Engineered from log returns derived in Main Area 1*
- EWMA realized volatility - *Computed from historical log returns using exponentially decaying weights*

> **Note:** Full implied volatility surfaces are not available via open-source channels for European indices. Volatility indices are therefore engineered for this project using realized returns data.

Main Area 3: Risk-Free Rate Term Structure - *Used for discounting option payoffs and defining the risk-neutral drift.*

- Zero-coupon yields by maturity - *Source: European Central Bank – Data Portal*

> **Note:** Annualized continuously compounded zero-coupon yields - published on an ACT/365 basis - across multiple maturities observed daily. Retrieval mode: JSON dumps from public URLs published by ECB. 

Main Area 4: Dividend Yield - *Used to correct forward price dynamics and maintain put–call parity consistency.*

- Index-level dividend yield (spot or trailing) - *Source: Yahoo Finance - EXSA.DE – index `dividendYield` field*

> **Note:** The dividend yield is treated as constant over time, consistent with standard equity option pricing practice.

Main Area 5: Option Chain Construction and Pricing - *Used for inter-model benchmarking, internal consistency checks, and analysis of model-implied smile and skew behavior.*

- Option type (Call / Put) - *Constructed using simple classification*
- Strike price grid (K) - *Constructed using volatility-scaled spot log-moneyness. $K = S_t * e^k$ where $k = L * \sigma_a * \sqrt(T)$*
- Time to maturity (T) - *Constructed - as per zero-coupon maturity dates*

>**Note:**
>To preserve economic relevance and numerical stability, the log-moneyness domain is capped between -0.7 and 0.7 while allowing volatility regimes to retain regime shocks and affect pricing within the admissible contract space. Without capping our k-values amplify mathematically (not economically) since $k \propto \sigma * \sqrt{T} * L$ hence, during high time to maturity and volatility grids (e.g., 2008 + 10Y) the k-values get mathematically amplified - which lead to Far OTM and Deep ITM strike prices - rendering price ~0, greek ~0 and all 3 models being numerically indistinguishable from one another. Hence, to avoid this mathematical noise, capping has been enforced.

> **Important Clarification:**  
> Under the proxy-based architecture, **no observed market option prices are sourced**. Option prices are fully **model-implied**, enabling controlled comparison of pricing dynamics across Black–Scholes–Merton, Heston, and Monte Carlo frameworks without contamination from microstructure noise or liquidity effects.


**1.4. Data Acquisition Sources and Rationale — ETF-Based Architecture**

The ETF-based architecture is introduced to provide **empirical pricing benchmarks** using observable option markets on highly liquid exchange-traded funds. While ETFs introduce tracking error and structural noise, their option chains offer the only feasible open-source alternative for observed option prices. Additionally, under the ETF-based architecture, the ETF itself is treated as the underlying tradable asset; index replication is not assumed.

The data points mirror those used in the proxy-based architecture to ensure methodological consistency; however, **data sources differ materially**.

Main Area 1: Underlying ETF Prices - *Used as the tradable underlying for observed option contracts.*

- Daily closing and adjusted prices - *Source: Yahoo Finance – EXSA.DE (iShares Core EURO STOXX 50 UCITS ETF, EUR, Xetra)*  
- Trading calendar - *Derived from ETF price series*

Main Area 2: Implied Volatility (Observed) - *Derived from observable option market prices for liquid ETFs and used for empirical validation.*

- Option-implied volatility (by strike and maturity) - *Extracted from Yahoo Finance ETF option chains and cleaned for liquidity and data quality*

Main Area 3: Risk-Free Rate Term Structure - *Used consistently across both architectures to maintain comparability.*

- Zero-coupon yields by maturity - *Source: ECB SDW API for EUR-denominated ETFs; FRED for USD-denominated ETFs*

Main Area 4: Dividend Yield - *Embedded in ETF price dynamics and option pricing.*

- ETF dividend yield - *Source: Yahoo Finance – ETF `dividendYield` field*

Main Area 5: Observed Option Chain Data - *Used for direct model-to-market pricing comparison and OOS evaluation.*

- Option type (Call / Put) - *Source: Yahoo Finance option chains*
- Strike price (K) - *Source: Yahoo Finance option chains*
- Expiration date and time to maturity (T) - *Source: Yahoo Finance option chains*
- Market option prices (mid, bid–ask) - *Source: Yahoo Finance option chains*
- Volume and open interest (where available) - *Source: Yahoo Finance option chains*


**Governance Note (SR 11-7 Alignment)**

The separation of proxy-based and ETF-based architectures ensures:

- Explicit documentation of data limitations and assumptions
- Clear distinction between **model development** and **empirical validation**
- Avoidance of overfitting or misrepresentation of model accuracy
- Traceability of all inputs to reproducible sources

This design aligns with SR 11-7 expectations regarding model transparency, validation independence, and controlled use of approximations.


**Section Output:**
At the end of the ETL process, the project produces separate market-state datasets and option-contract datasets for both proxy-based and ETF-based architectures. The proxy datasets support controlled model development and assumption testing, while the ETF datasets enable empirical validation against observed option prices. Model prices generated under the ETF architecture are compared against market option prices—not against proxy-generated prices—to ensure economically meaningful validation and SR 11-7-compliant separation of development and benchmarking.

In [25]:
def pull_zero_coupon_data():
    url = [
        'https://data-api.ecb.europa.eu/service/data/YC/B.U2.EUR.4F.G_N_A.SV_C_YM.SR_3M?startPeriod=2008-04-04&endPeriod=2026-01-30&format=jsondata',
        'https://data-api.ecb.europa.eu/service/data/YC/B.U2.EUR.4F.G_N_A.SV_C_YM.SR_6M?startPeriod=2008-04-04&endPeriod=2026-01-30&format=jsondata',
        'https://data-api.ecb.europa.eu/service/data/YC/B.U2.EUR.4F.G_N_A.SV_C_YM.SR_1Y?startPeriod=2008-04-04&endPeriod=2026-01-30&format=jsondata',
        'https://data-api.ecb.europa.eu/service/data/YC/B.U2.EUR.4F.G_N_A.SV_C_YM.SR_2Y?startPeriod=2008-04-04&endPeriod=2026-01-30&format=jsondata',
        'https://data-api.ecb.europa.eu/service/data/YC/B.U2.EUR.4F.G_N_A.SV_C_YM.SR_5Y?startPeriod=2008-04-04&endPeriod=2026-01-30&format=jsondata',
        'https://data-api.ecb.europa.eu/service/data/YC/B.U2.EUR.4F.G_N_A.SV_C_YM.SR_10Y?startPeriod=2008-04-04&endPeriod=2026-01-30&format=jsondata'
    ]
    data_dump_yield = []
    data_dump_dates = []
    data_dump_ids = []
    df_zero_coupon = pd.DataFrame()
    for i in url:
        req = requests.get(i, timeout=60)
        req.raise_for_status()
        json_dump = req.json()
        yield_data = json_dump['dataSets'][0]['series']['0:0:0:0:0:0:0']['observations']
        for i in range(len(yield_data)):
            data_dump_yield.append(yield_data[str(i)][0])
            id_data = json_dump['structure']['dimensions']['series'][6]['values'][0]['id']
            data_dump_ids.append(id_data)

        time_data = json_dump['structure']['dimensions']['observation'][0]['values']
        for i in range(len(time_data)):
            data_dump_dates.append(time_data[i]['name'])

    
    df_zero_coupon['yields'] = data_dump_yield
    df_zero_coupon['yields'] = df_zero_coupon['yields'] / 100
    df_zero_coupon['Date'] = data_dump_dates
    df_zero_coupon['ID'] = data_dump_ids
    df_zero_coupon['Date'] = pd.to_datetime(df_zero_coupon['Date'])
    df_zero_coupon = (
        df_zero_coupon
        .pivot(index='Date', columns='ID', values='yields')
        .reset_index()
    )
    df_zero_coupon = df_zero_coupon.set_index('Date')
    df_zero_coupon.to_csv(zero_coupon)
    return df_zero_coupon

In [26]:
# Proxy-based architecture implementation
def proxy_architecture_data_generation():
    # 1. Index closing prices
    df_proxy_index = yf.download('EXSA.DE', start='2008-01-01', end='2026-02-01', progress=False)
    df_proxy_index = (
        df_proxy_index
        .xs('Close', level='Price', axis=1)
        .rename(columns={'EXSA.DE': 'index_closing_price'})
        .sort_index()
    )

    # 3. Zero-coupon yield
    with time_block('Zero Coupon Data Load'):
        df_zero_coupon = pull_zero_coupon_data()

    df_proxy_index = df_proxy_index.join(
        df_zero_coupon,
        how='inner'
    )

    # 4. Dividend-yield
    exsa = yf.Ticker('EXSA.DE')
    div = exsa.info.get('dividendYield') / 100


    # Performing data consolidation, engineering and sanity checks
    # Data engineering
    df_proxy_index['index_closing_simple_ret'] = df_proxy_index['index_closing_price'].pct_change()
    df_proxy_index['index_closing_log_ret'] = np.log(1 + df_proxy_index['index_closing_simple_ret'])
    # 2. Volatility proxies (calculated in this section to avoid data conflicts)
    lam = 0.94
    df_proxy_index['daily_rol_vol_21D'] = df_proxy_index['index_closing_log_ret'].rolling(21).std(ddof=1)
    df_proxy_index['daily_rol_vol_63D'] = df_proxy_index['index_closing_log_ret'].rolling(63).std(ddof=1)
    df_proxy_index['daily_ewma_vol'] = np.sqrt(df_proxy_index['index_closing_log_ret'].pow(2).ewm(alpha=1-lam, adjust=False).mean()) # set adjust = False to compute volatility using the classical EWMA vol model. True will use the statistical definition.
    df_proxy_index['dividend_yield'] = div
    df_proxy_index = df_proxy_index.dropna(how='any')
    # 5. Option chain construction (implemented here to preserve data structure)
    # Maturity dates construction
    T_in_years = [0.25, 0.50, 1.00, 2.00, 5.00, 10.00]
    df_proxy_index = (
        df_proxy_index
        .loc[df_proxy_index.index.repeat(len(T_in_years))]
        .assign(T_in_years=T_in_years * len(df_proxy_index))
        .set_index('T_in_years', append=True)
        .sort_index()
    )
    # Strike price construction
    L_grid = np.array([-2.5,-2.0,-1.5,-1.0,-0.5,0,0.5,1.0,1.5,2.0,2.5])
    n = len(df_proxy_index)
    df_proxy_index = (
        df_proxy_index
        .loc[df_proxy_index.index.repeat(len(L_grid))]
    )
    df_proxy_index['L'] = np.tile(L_grid, n)
    df_proxy_index['annualized_ewma_vol'] = df_proxy_index['daily_ewma_vol'].to_numpy() * np.sqrt(252)
    k_raw = df_proxy_index['L'] * df_proxy_index['annualized_ewma_vol'] * np.sqrt(df_proxy_index.index.get_level_values('T_in_years'))
    df_proxy_index['k'] = np.clip(k_raw, -0.7, 0.7)
    df_proxy_index['K'] = df_proxy_index['index_closing_price'] * np.exp(df_proxy_index['k'])
    # Call/ Put classification construction
    c_p_classification = ['C', 'P']
    df_proxy_index = (
        df_proxy_index
        .loc[df_proxy_index.index.repeat(2)]
    )
    df_proxy_index['call_put_classification'] = np.tile(c_p_classification, len(df_proxy_index)//2)
    df_class_check = (
        df_proxy_index.groupby([
            pd.Grouper(level='Date'),
            pd.Grouper(level='T_in_years'),
            'K'
        ])['call_put_classification']
        .nunique()
    )
    mismatch = df_class_check[df_class_check < 2]
    if len(mismatch) > 0:
        print(f'{len(mismatch)} records with incorrect classification structure found.')
    else:
        print('Classification structure complete.')

    # Data sanity checks
    assert df_proxy_index.index.is_monotonic_increasing
    assert (df_proxy_index['index_closing_simple_ret'] > -1).all()

    # Dropping duplicates and creating final master
    df_proxy_index_master = (
        df_proxy_index
        .reset_index()
        .drop_duplicates(subset=['Date', 'T_in_years', 'L', 'call_put_classification'], keep='first')
        .sort_values(['Date', 'T_in_years', 'L', 'call_put_classification'])
        .set_index(['Date', 'T_in_years'])
    )
    
    # Configuring data display template
    df_proxy_index_master.style.set_table_styles(
        [
            {
                'selector': 'th',
                'props': [('text-align','center')]
            },
            {
                'selector': 'td',
                'props': [('text-align', 'right')]
            }
        ]
    )

    # Diagnostic print statements
    print('Data sourced for index prices:')
    display(df_proxy_index_master.head(44))
    print(f'Total number of records in master proxy data: {len(df_proxy_index_master)}')
    
    # Uploading data to pickle file
    df_proxy_index_master.to_pickle(proxy_master)

In [27]:
# Calling data generation programs
# with time_block('Proxy data generation block'):
#     proxy_architecture_data_generation()

In [28]:
# ETF-based architecture implementation
# 1. Index closing prices
df_etf_index = yf.download('EXSA.DE', start='2008-01-01', end='2026-02-01', progress=False)
df_etf_index = (
    df_etf_index
    .xs('Close', level='Price', axis=1)
    .rename(columns={'EXSA.DE': 'index_closing_price'})
    .sort_index()
)

print(df_etf_index.head())

Ticker      index_closing_price
Date                           
2008-01-02              24.6418
2008-01-03              24.5673
2008-01-04              24.0930
2008-01-07              24.0456
2008-01-08              24.1879
